# Analysis of the historical price of a troy ounce of silver

## Setup

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from scipy import signal
from statsmodels.tsa.seasonal import STL

In [ ]:
# Get silver price df
df_ag = pd.read_csv("../data/AG.csv")

In [ ]:
# Set date as index of the df
df_ag["date"] = pd.to_datetime(df_ag["date"])
df_ag.set_index("date", inplace=True)

In [ ]:
# Set charts theme
sns.set_theme(style="darkgrid", rc={"grid.alpha": 0.33})
plt.style.use("dark_background")

In [ ]:
# Save chart as png function
def save_chart_as_png(filename: str) -> None:
    plt.savefig(
        f"../images/{filename}.png",
        format="png",
        dpi=300,
        orientation="landscape",
        bbox_inches="tight",
    )

## Dataset basic info

In [ ]:
# First and last entries
pd.concat([df_ag.head(1), df_ag.tail(1)]).T

In [ ]:
df_ag.describe().T

In [ ]:
# How many trading days per year on average
days_per_year = df_ag[df_ag.index.year != 2024].index.year.value_counts()
days_per_year.mean().round(2)

## Price of silver across time (long-term analysis)

In [ ]:
# Get 1-year moving average (252 trading days per year)
df_ag["price_1y_ma"] = df_ag["price"].rolling(window=252).mean()

In [ ]:
plt.figure(figsize=(10, 6))

sns.lineplot(data=df_ag, x=df_ag.index, y="price", color="lightgrey", linewidth=0.25)
sns.lineplot(data=df_ag, x=df_ag.index, y="price_1y_ma", label="Moving average (1-year)", color="aqua", linewidth=0.75)

plt.title("Price of a troy ounce of silver across time")
plt.xlabel("")
plt.ylabel("")

plt.show()

**It's easier to see the early price fluctuations with a logarithmic scale on the y-axis**

In [ ]:
plt.figure(figsize=(10, 6))

sns.lineplot(data=df_ag, x=df_ag.index, y="price", color="lightgrey", linewidth=0.25)
sns.lineplot(data=df_ag, x=df_ag.index, y="price_1y_ma", label="Moving average (1-year)", color="aqua", linewidth=0.75)

# Compress the y axis to see early price fluctuations
plt.yscale("log")

plt.title("Price of a troy ounce of silver across time")
plt.xlabel("")
plt.ylabel("")

plt.show()

In [ ]:
# All-time high
ath_date = df_ag["price"].idxmax()
df_ag.loc[[ath_date]]

In [ ]:
# All-time low
atl_date = df_ag["price"].idxmin()
df_ag.loc[[atl_date]]

In [ ]:
# Peaks
peaks, _ = signal.find_peaks(df_ag["price"], distance=1000)
df_ag.iloc[peaks].nlargest(12, "price").sort_values("date")[["price"]].T

In [ ]:
# Valleys
valleys, _ = signal.find_peaks(-df_ag["price"], distance=1000)
df_ag.iloc[valleys].nsmallest(12, "price").sort_values("date")[["price"]].T

In [ ]:
# Price appreciation since first entry
first_entry_price = df_ag.iloc[0]["price"]
last_entry_price = df_ag.iloc[-1]["price"]
(last_entry_price - first_entry_price) / first_entry_price

**Key takeaways:**
- Silver hit its historic low in 1970, hovering around \$1.3.
- Similar to gold, its value surged swiftly from the early 1970s.
- The pinnacle was reached in 1980, soaring to \$49.5.
- Following this peak, a period of decline ensued until the early 1990s, stabilizing thereafter until the early 2000s.
- Subsequently, there was a consistent ascent until the early 2010s, culminating in a peak of approximately \$48.7 in 2011.
- Despite some fluctuations, there was a general downtrend and stabilization until the onset of the pandemic, after which a renewed ascent began.
- The price has appreciated by approximately 1,270% since its initial point.

### Price change year-over-year

In [ ]:
# Get YoY returns
# Get yearly silver price df with first and last prices
df_ag_yearly = df_ag.groupby(df_ag.index.year)["price"].agg(
    first_price="first",
    last_price="last"
)
# Get YoY return
df_ag_yearly["price_change"] = (df_ag_yearly["last_price"] - df_ag_yearly["first_price"]) / df_ag_yearly["first_price"]

In [ ]:
plt.figure(figsize=(10, 6))

sns.barplot(data=df_ag_yearly, x=df_ag_yearly.index, y="price_change", color="silver")

# Compress the y axis to see smaller bars
plt.yscale("symlog")

plt.title("Year-over-year return of silver across time")
plt.xlabel("")
plt.ylabel("")

# Show only beginning of decade
ax = plt.gca()
for index, label in enumerate(ax.get_xticklabels()):
    if index % 10 - 2 != 0:
        label.set_visible(False)

plt.show()

In [ ]:
# Highest YoY return
df_ag_yearly.loc[[df_ag_yearly["price_change"].idxmax()]]

In [ ]:
# Lowest YoY return
df_ag_yearly.loc[[df_ag_yearly["price_change"].idxmin()]]

In [ ]:
# Average YoY return
df_ag_yearly["price_change"].mean().round(4)

In [ ]:
# Average YoY return excluding 1979
df_ag_yearly.loc[df_ag_yearly.index != 1979,"price_change"].mean().round(4)

In [ ]:
# Median YoY return
df_ag_yearly["price_change"].median().round(4)

In [ ]:
# Standard deviation YoY return
df_ag_yearly["price_change"].std().round(4)

In [ ]:
# Cumulative product YoY return
(1 + df_ag_yearly["price_change"]).cumprod().iloc[-1].round(4) - 1

In [ ]:
# Get max, min, average, median, standard deviation and cumulative product YoY return per decade
df_ag_yearly["decade"] = (df_ag_yearly.index // 10) * 10
df_ag_yearly.groupby("decade").agg(
    max_yoy_price_change=("price_change", lambda x: x.max().round(4)),
    min_yoy_price_change=("price_change", lambda x: x.min().round(4)),
    avg_yoy_price_change=("price_change", lambda x: x.mean().round(4)),
    median_yoy_price_change=("price_change", lambda x: x.median().round(4)),
    std_yoy_price_change=("price_change", lambda x: x.std().round(4)),
    cumprod_yoy_price_change=("price_change", lambda x: (1 + x).cumprod().iloc[-1].round(4) - 1),
    dod_price_change=("decade", lambda x: (df_ag_yearly.loc[x.index, "last_price"].iloc[-1] - df_ag_yearly.loc[x.index, "first_price"].iloc[0]) / df_ag_yearly.loc[x.index, "first_price"].iloc[0])
).T

**Key takeaways:**
- In 1979, silver experienced an astonishing surge, with a remarkable 430% increase.
- However, the following year, it plummeted, marking the lowest return at approximately -61.2%.
- Over the years, silver has shown an average year-over-year return of approximately 11.8%, surpassing gold. Yet, the median return stands at -1.5%.
- When 1979 is ignored, the average YoY drops significantly to about 4.4%.
- Mirroring gold's performance, the 1970s emerged as the standout decade for silver, boasting an average YoY return of around 58%, with a much lower median.
- The 1980s, in contrast, witnessed a downturn, with an average return of approximately -14%.
- Transitioning into the 2000s, the average return was approximately 15%.
- As we go through the current decade, silver continues to show promise with an average return of around 11.8%. However, the median return hovers around 4.6%.

### Yearly volatility

In [ ]:
# Get daily price change
df_ag["price_change"] = df_ag["price"].pct_change()

In [ ]:
# Biggest price changes
df_ag.loc[df_ag["price_change"].abs().sort_values(ascending=False).head(10).index, ["price_change"]].T

In [ ]:
# Get 1-year moving standard deviation
df_ag["volatility_1y"] = df_ag["price_change"].rolling(window=252).std()

In [ ]:
plt.figure(figsize=(10, 6))

sns.lineplot(data=df_ag,x=df_ag.index,y="volatility_1y", color="red", linewidth=0.75)

plt.title("Yearly volatility of the price of silver across time")
plt.xlabel("")
plt.ylabel("")

plt.show()

In [ ]:
# All-time high
ath_date = df_ag["volatility_1y"].idxmax()
df_ag.loc[[ath_date], ["price", "volatility_1y"]]

In [ ]:
# All-time low
atl_date = df_ag["volatility_1y"].idxmin()
df_ag.loc[[atl_date], ["price", "volatility_1y"]]

In [ ]:
# Top 5 peaks
peaks, _ = signal.find_peaks(df_ag["volatility_1y"], distance=500)
df_ag.iloc[peaks].nlargest(5, "volatility_1y").sort_values("date")[["volatility_1y"]].T

In [ ]:
# Top 5 valleys
valleys, _ = signal.find_peaks(-df_ag["volatility_1y"], distance=500)
df_ag.iloc[valleys].nsmallest(5, "volatility_1y").sort_values("date")[["volatility_1y"]].T

In [ ]:
# Average 1-year volatility
df_ag["volatility_1y"].mean().round(3)

In [ ]:
# Average 1-year volatility per decade
df_ag_dec = df_ag.groupby((df_ag.index.year // 10) * 10)
df_ag_dec = df_ag_dec["volatility_1y"].mean().round(3).reset_index()
df_ag_dec.columns = ["decade", "average_volatility_1y"]
df_ag_dec.set_index("decade").T

**Key takeaways:**
- The 1980s witnessed the largest daily price fluctuations.
- Over the years, the average yearly volatility has been 2%.
- The all-time high was in in 1980, and the all-time low in 2001.
- Periods of heightened volatility occurred during the mid-70s and early 80s, and also during major economic events such as the Great Recession, EU sovereign debt crisis, and the pandemic.
- Conversely, the market was very stable during the 90s, early 2000s, and late 2010s.

### STL decomposition (trend, seasonality, and residuals)

In [ ]:
stl = STL(df_ag["price"], period=252).fit()

In [ ]:
fig, axes = plt.subplots(4, 1, figsize=(10, 6), sharex=True)

axes[0].plot(df_ag.index, df_ag["price"], label="Original", color="lightgrey", linewidth=0.5)
axes[0].set_title("Price of a troy ounce of silver across time")

axes[1].plot(df_ag.index, stl.trend, label="Trend", color="aqua", linewidth=1)
axes[1].set_title("Trend component")

axes[2].plot(df_ag.index, stl.seasonal, label="Seasonal", color="fuchsia", linewidth=0.5)
axes[2].set_title("Seasonal component")

axes[3].plot(df_ag.index, stl.resid, label="Residual", color="orangered", linewidth=0.5)
axes[3].set_title("Residual component")

plt.tight_layout()
plt.show()

#### Trend analysis

In [ ]:
plt.figure(figsize=(10, 6))

plt.plot(stl.trend, color="aqua", linewidth=1)

plt.title("Trend component of the price of silver across time")
plt.xlabel("")
plt.ylabel("Trend")

plt.yscale("log")

plt.show()

**Key takeaways:**
- The trend grew significantly throughout the 1970s.
- Following this peak, there was a gradual decline until the 1990s.
- After that, there was a resurgence from the early 2000s until the early 2010s.
- There's 30-year valley spanning from the 1980s to the 2010s.

#### Seasonality analysis

In [ ]:
seasonal = stl.seasonal
monthly_avgs = seasonal.groupby(seasonal.index.month).mean()

plt.figure(figsize=(10, 6))

plt.plot(monthly_avgs.index, monthly_avgs.values, marker='o', color="fuchsia", linewidth=1)

plt.title("Average of the seasonal component of price of silver over the year")
plt.xlabel("Month")
plt.ylabel("Seasonality")

plt.show()

**Key takeaways:**
- Seasonality has variations across different years.
- Just like gold, based on monthly averages, there's favorable seasons during the first four months of the year, with an additional peak in September.

## Price of silver year-to-date (short-term analysis)

In [ ]:
# Get YTD df
df_ag_ytd = df_ag["2024":].copy()

In [ ]:
plt.figure(figsize=(10, 6))

sns.lineplot(data=df_ag_ytd, x=df_ag_ytd.index, y="price", color="lightgrey", linewidth=1)

plt.xticks(fontsize=8)

plt.title("Price of a troy ounce of silver since 2024")
plt.xlabel("")
plt.ylabel("")

plt.show()

In [ ]:
# Price change YTD
first_price = df_ag_ytd.iloc[0]["price"]
last_price = df_ag_ytd.iloc[-1]["price"]
(last_price - first_price) / first_price

In [ ]:
# When the price quickly began to rise
rise_start = df_ag_ytd.loc[df_ag_ytd["price"].diff() > 0.75].index[0]
df_ag_ytd.loc[[rise_start], ["price", "price_change"]].round(2)

In [ ]:
# When the price sort of peaked
peak_date = df_ag_ytd[:"2024-05-01"]["price"].idxmax()
df_ag_ytd.loc[[peak_date], ["price", "price_change"]].round(2)

In [ ]:
# Average daily price change during the rise
df_ag_ytd.loc[rise_start:peak_date]["price_change"].mean().round(5)

In [ ]:
# Average price before the rise
avg_price_before = df_ag_ytd.loc[:rise_start]["price"].mean().round(0)
avg_price_before

In [ ]:
# Average price after the peak
avg_price_after = df_ag_ytd.loc[peak_date:]["price"].mean().round(0)
avg_price_after

In [ ]:
# Difference between the average prices
((avg_price_after - avg_price_before) / avg_price_before).round(3)

**Key takeaways:**
- The year-to-date price change stands at approximately 24%.
- Prior to the breakout, the average price lingered around \$23.
- A notable price breakout occurred in early March.
- Post-breakout, the price exhibited an average daily increase of about 0.9%.
- The price peaked around \$29 in mid-April.
- Since reaching its peak, the average price has stabilized around \$28.

### Weekly volatility

In [ ]:
# Get 1-week moving standard deviation (5 trading days per week)
df_ag_ytd["volatility_1w"] = df_ag_ytd["price_change"].rolling(window=5).std()

In [ ]:
plt.figure(figsize=(10, 6))

sns.lineplot(data=df_ag_ytd, x=df_ag_ytd.index, y="volatility_1w", color="red", linewidth=1)

plt.xticks(fontsize=8)

plt.title("Weekly volatility of the price of silver across time")
plt.xlabel("")
plt.ylabel("")

plt.show()

In [ ]:
# Average volatility
df_ag_ytd["volatility_1w"].mean().round(4)

**Key takeaways:**
- Average weekly volatility has been gradually increasing over time.
- Unlike gold, silver has not shown any clear breakout trends.